# 모델 경량화 기법 – Pruning & Quantization

## Day 28: 모델 경량화 기법 – Pruning & Quantization

---

### 1) Theory: 왜 경량화가 필요한가?

* **실제 서비스 제약**

  * 모바일·IoT 기기, 엣지 디바이스에서는 연산·메모리 예산이 제한적
  * 모델 크기·추론 속도를 줄여야 사용자 경험(배터리·응답 속도)을 지킬 수 있음
* **Pruning (가지치기)**

  * **Unstructured Pruning**: 개별 가중치(파라미터) 중 작은 값(절댓값 기준)을 0으로 강제

    * → 희소성(sparsity)을 높여, 저장 공간·메모리 접근량 절감
  * **Structured Pruning**: 채널(channel)·필터(filter) 단위로 통째로 제거

    * → 실제 연산량(Flops) 감소 효과가 뚜렷
* **Quantization (양자화)**

  * **Post-Training Quantization** (PTQ): 학습 후 `float32` → `int8` 등으로 변환
  * **Quantization-Aware Training** (QAT): 양자화 오차를 학습 과정에 반영하여 성능 저하 최소화
  * 숫자 표현 비트를 줄여 메모리 절감, 벡터 연산 가속

#### 실행 결과 예상

* **가지치기** 후: 약간의 정확도 감소(±1% 이내)
* **양자화** 후: 정확도 큰 손실 없이(±2% 이내) 파라미터 크기 절반 이하로 감소

---

### 3) 심화 과제

1. **Structured Pruning** (채널 단위) 해 보기

   ```python
   prune.ln_structured(model.fc1, name='weight', amount=0.2, n=2, dim=0)
   ```
2. **Post-Training Static Quantization**

   * `quant.prepare` → **Calibration Data** 통과 → `quant.convert`
3. **QAT**

   * `quant.prepare_qat` → 학습 진행 → `quant.convert`
4. **모델 비교 보고서**

   * 원본 vs Pruned vs Quantized 성능·크기·추론 속도 비교



In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn.utils import prune as prune
from torch import quantization as quant
from torch.utils.data import DataLoader as loader
from torchvision import datasets, transforms 

In [9]:
class Models(nn.Module):
    def __init__(self):
        super(Models,self).__init__()
        self.fc1=nn.Linear(28*28,256)
        self.fc2=nn.Linear(256, 10)

    def forward(self, x):
        x=x.view(x.size(0),-1)
        x=torch.relu(self.fc1(x))
        out=self.fc2(x)
        return out
    

In [13]:
transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_data=datasets.MNIST('.',train=True, download=True,transform=transform)
test_data=datasets.MNIST('.',train=False,download=True,transform=transform)

trian_loader=loader(train_data, batch_size=256, shuffle=True)
test_loader=loader(test_data, batch_size=1000, shuffle=True)

In [15]:
device=torch.device('cuda'if torch.cuda.is_available()else'cpu')
print(device,'mode')
model=Models().to(device)

cuda mode


In [17]:
def evalmode(model):
    model.eval()
    correct=0
    with torch.no_grad():
        for data, target in test_loader:
            data,target=data.to(device), target.to(device)
            out=model(data)
            correct+=(out.argmax(1)==target).sum().item()
        return correct/len(test_loader.dataset)

In [21]:
opt=optim.SGD(model.parameters(),lr=0.01)
criter=nn.CrossEntropyLoss()
model.train()
data,target=next(iter(trian_loader))
data,target=data.to(device),target.to(device)
opt.zero_grad()
out=model(data)
loss=criter(out,target)
loss.backward()
opt.step()
print("원본 모델 정확도:", evalmode(model))

원본 모델 정확도: 0.1259


In [27]:
# — 5) Unstructured Pruning 적용 (20% 희소화) —
prune.random_unstructured(model.fc1,name='weight',amount=0.2)
prune.random_unstructured(model.fc2,name='weight',amount=0.2)

print("Pruned 모델 정확도:", evalmode(model))


Pruned 모델 정확도: 0.1468


In [33]:
'''qmodel=quant.quantize_dynamic(
    model,{nn.Linear}, dtype=torch.qint8
).to(device)'''
qmodel = quant.quantize_dynamic(
    model, {nn.Linear}, dtype=torch.qint8
).to(device)
print("Quantized 모델 정확도:", evalmode(qmodel))


AttributeError: 'Linear' object has no attribute 'weight_mask'

In [50]:
# (1) model이 GPU에 올라가 있었다면 CPU로 꺼내고
model = model.cpu()

# (2) prune 훅을 제거하고
prune.random_unstructured(model.fc1, 'weight', amount=0.2)
prune.random_unstructured(model.fc2, 'weight', amount=0.2)
prune.remove(model.fc1, 'weight')
prune.remove(model.fc2, 'weight')

# (3) 동적 양자화 역시 CPU에서만 적용
qmodel = torch.quantization.quantize_dynamic(
    model,
    {torch.nn.Linear},
    dtype=torch.qint8
)   # 이미 qmodel은 CPU 상의 모델

# (4) 이제 평가 함수도 CPU 전용으로
def eval_cpu(m):
    m.eval()
    correct = 0
    with torch.no_grad():
        for x, y in test_loader:
            # test_loader가 GPU가 아니라 CPU 데이터를 준다고 가정
            out = m(x)  
            correct += (out.argmax(1) == y).sum().item()
    return correct / len(test_loader.dataset)

print("Quantized 모델 정확도:", eval_cpu(qmodel))


Quantized 모델 정확도: 0.0949


In [52]:
import torch.backends.quantized as qb
qb.engine = 'fbgemm'   # x86 CPU에서 양자화용 연산을 수행할 백엔드


In [54]:
# — 7) 크기 비교 —
import io
def size_of(m):
    buf = io.BytesIO()
    torch.save(m.state_dict(), buf)
    return buf.getbuffer().nbytes / 1e6  # MB

print(f"원본 파라미터 크기: {size_of(model):.3f} MB")
print(f"Quantized 파라미터 크기: {size_of(qmodel):.3f} MB")


원본 파라미터 크기: 0.816 MB
Quantized 파라미터 크기: 0.207 MB
